In [ ]:
import polars as pl
import numpy as np
import subprocess
import os
os.chdir('/zata/zippy/ramirezc/splice-model-benchmark/golden_standard')
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
chroms = [f"chr{x}" for x in range(1, 23)] + ["chrX", "chrY"]
shortread_introns_df = pl.read_csv("all_shortread_introns.tab", separator="\t").with_columns(pl.col("chrom").cast(pl.Enum(chroms))).sort("chrom", "start", "strand")
shortread_introns_df.with_columns(pl.lit(".").alias("score"), pl.lit(".").alias("name")).select("chrom", "start", "end", "name", "score", "strand").write_csv("all_shortread_introns.bed", separator="\t", include_header=False)
shortread_introns_standardized = shortread_introns_df.with_columns(
    ((pl.col("reads") - pl.col("reads").mean()) / pl.col("reads").std()).alias("reads_standardized")
)
display(shortread_introns_standardized)

longread_introns_df = pl.read_csv("all_longread_introns.tab", separator="\t").with_columns(pl.col("chrom").cast(pl.Enum(chroms))).sort("chrom", "start", "strand")
longread_introns_df.with_columns(pl.lit(".").alias("score"), pl.lit(".").alias("name")).select("chrom", "start", "end", "name", "score", "strand").write_csv("all_longread_introns.bed", separator="\t", include_header=False)
longread_introns_standardized = longread_introns_df.with_columns(
    ((pl.col("reads") - pl.col("reads").mean()) / pl.col("reads").std()).alias("reads_standardized")
)
display(longread_introns_standardized)

In [ ]:
display(shortread_introns_standardized.select('reads', 'reads_standardized').describe())
display(longread_introns_standardized.select('reads', 'reads_standardized').describe())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

plt.style.use('cowplot')
fig.suptitle('Standard Sclaed Read Distribution for long and short read RNA-seq datasets', fontsize=16)
fig.supxlabel('Standard Scaled Reads')
fig.supylabel('Frequency')
axes[0].hist(np.log2(longread_introns_standardized['reads_standardized']), bins=200, alpha=0.5, label='Read Length')
axes[0].set_title('Long Read RNA-seq')
axes[1].hist(np.log2(shortread_introns_standardized['reads_standardized']), bins=200, alpha=0.5, label='Read Length')
axes[1].set_title('Total Read RNA-seq')
for axis in axes:
    y_ticks = axis.get_yticks()
    padding = y_ticks[1] * 0.1
    y_lim_min = y_ticks[0] - padding
    axis.set_ylim(bottom=y_lim_min)
plt.show()



In [ ]:
cmd = [
    "bedtools", "jaccard", 
    "-a", "all_shortread_introns.bed",
    "-b", "all_longread_introns.bed"
]
jaccard_stats = subprocess.run(cmd, capture_output=True, text=True)
print(jaccard_stats.stdout)

intersection	union	jaccard	n_intersections
2039488805	2531104536	0.80577	65046



Strand specific jaccard is not right, it seems that the standedness is being mixed up between the different platforms and interpretations of the data from pysam. I'm going to manually parse the BAM flags to get the most accuracte strand information.

In [42]:
import pysam

file_prefix = "/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/{biosample}/{file}.filtered.bam"
metadata = pl.read_csv("/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/biosample_matched_rna_seq_experiments.tsv", separator="\t")

file_paths = []
for row in metadata.iter_rows(named=True):
    file_acc = row["File accession"]
    biosample_acc = row["Biosample accession"]
    file_paths.append(file_prefix.format(biosample=biosample_acc, file=file_acc))

def is_paired_end(bam_path):
    """Check if a BAM file is paired-end by examining the first 1000 reads"""
    with pysam.AlignmentFile(bam_path, "rb") as bam:
        for i, read in enumerate(bam):
            if read.is_paired:
                return True
            if i >= 1000:
                break
        return False

for bam_file in file_paths:
    paired = is_paired_end(bam_file)
    if paired is True:
        print(f"{bam_file}: Paired-end")
    elif paired is False:
        print(f"{bam_file}: Single-end")
    else:
        print(f"{bam_file}: Error checking file")


/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS481WHG/ENCFF068AID.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS660AUE/ENCFF791RZL.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS422TMB/ENCFF083LBA.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS644WSW/ENCFF808UYE.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS105DDE/ENCFF635SRJ.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS819UHF/ENCFF349PBM.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS481WHG/ENCFF564ONS.filtered.bam: Single-end
/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS914REQ/ENCFF502LAB.filtered.bam: Sin

In the future when working with stranded data (forward, reverse), know that it will affect how the BAM files are parsed. For example in a pysam iterator:

```python
for chrom in chroms:
    for read in samfile.fetch(chrom):
        if library_type == 'forward':
            return '-' if read.is_reverse else '+'
        elif library_type == 'reverse':
            return '+' if read.is_reverse else '-'
```

For unstranded data, you cannot determine the strand of the read, so the data must remain strand agnostic.

Count number of BAM files coming from unstranded libraries

In [6]:
metadata = pl.read_csv("/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/biosample_matched_rna_seq_experiments.tsv", separator="\t")
display(metadata.select("Assay", "Library strand specific").to_struct().value_counts())

,count
struct[2],u32
"{""long read RNA-seq"",""unstranded""}",69
"{""total RNA-seq"",""reverse""}",95
"{""long read RNA-seq"",""forward""}",32
